# Build Data Duration Hedge CRSP

In [1]:
DATE = '2024-10-31'

In [2]:
import pandas as pd
import numpy as np
from datetime import date

In [3]:
import sys
sys.path.insert(0, '../cmds')
from treasury_cmds import process_wrds_treasury_data

In [4]:
import wrds
file_key = open("../../keys/wrds_username.txt","r")
USERNAME = file_key.read()
file_key.close()

db = wrds.Connection(wrds_username=USERNAME)
#db.create_pgpass_file()

Loading library list...
Done


***

# Get Timeseries for Duration Analysis

Update the KEYNOs. 
* These three are issued in Aug 2019.
* Idea is 5, 10, 30 maturities.
* 5-year matures in Aug 2024. So will need updated.

Used in HW 2, problem 3.
* Only make use of the 30yr and 10yr there.

In [5]:
KEYNOS = (207392, 207391)

SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where kytreasno in {KEYNOS}'
data_quotes = db.raw_sql(SQL_QUERY)

SQL_QUERY = f'select * from crsp_m_treasuries.tfz_iss where kytreasno in {KEYNOS}'
data_iss = db.raw_sql(SQL_QUERY)

rawdata = data_quotes.merge(data_iss, on='kytreasno')
data = process_wrds_treasury_data(rawdata,keys_extra=['duration'])

In [6]:
px = data.pivot_table(index='quote date',columns='KYTREASNO',values='price').dropna()
duration = data.pivot_table(index='quote date',columns='KYTREASNO',values='duration').dropna()

***

# Save to Excel

In [8]:
outfile = f'../data/treasury_ts_duration_{DATE}.xlsx'
with pd.ExcelWriter(outfile) as writer:      
    data.to_excel(writer, sheet_name= 'database', index=True)
    px.to_excel(writer, sheet_name= 'price', index=True)
    duration.to_excel(writer, sheet_name= 'duration', index=True)